In [11]:
import pandas as pd
import google.generativeai as genai
import time

# Configure the Gemini API
genai.configure(api_key="Replace with your API")
model = genai.GenerativeModel('gemini-2.0-flash-001') # Using the flash model

# Load the Excel file
file_path = 'Starbucks comment.xlsx'
df = pd.read_excel(file_path)

# Create a new column 'likes_starbucks'
df['likes_starbucks'] = None

In [ ]:
# Function to analyze sentiment with Gemini
def analyze_sentiment(comment):
    if pd.isna(comment):
        return "N/A"

    prompt = f"identify the text and identify whether the consumer like Starbuck or not. return yes or no.\n\nText: \"{comment}\""

    try:
        response = model.generate_content(prompt)
        result = response.text.strip().lower()

        # Extract just the yes/no answer
        if "yes" in result.lower():
            return "Yes"
        elif "no" in result.lower():
            return "No"
        else:
            return "Unclear"
    except Exception as e:
        print(f"Error analyzing comment: {e}")
        return "Error"

# Process each comment with rate limiting to avoid API throttling
for i, row in df.iterrows():
    print(f"Processing comment {i+1}/{len(df)}")
    df.at[i, 'likes_starbucks'] = analyze_sentiment(row['comment'])

    # Sleep to avoid hitting API rate limits (adjust as needed)
    if i % 10 == 0 and i > 0:
        time.sleep(0.5)

# Save the updated dataframe to a new Excel file
df.to_excel('Starbucks_comments_analyzed.xlsx', index=False)

print("Analysis complete! Results saved to 'Starbucks_comments_analyzed.xlsx'")